In this file I am preparing the data from processing and retrieving audio features and artist genres using the spotify API.

To fetch the audio features and genres for each song I call upon functions imported from my script spotify_api_functions that allows me to create a spotify client object and make API calls while keeping my secret and cid stored in a gitignored .env file.
After running the cells below I will have the following .csv files to be used in my analysis.
- artists_with_genres.csv: df containing all of the artists I have listened to and their associated genres and total minutes played
- songs_with_audio_features: df containing all of the songs I have listened to and audio features and total minutes played
- stream.csv: df containing all of my instances of listening to songs organized by the date and time listened


In [4]:
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
import matplotlib.pyplot as plt
import operator
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import time
import math
import spotify_api_functions as spotify

{'display_name': 'grantdidway', 'external_urls': {'spotify': 'https://open.spotify.com/user/grantdidway12345'}, 'href': 'https://api.spotify.com/v1/users/grantdidway12345', 'id': 'grantdidway12345', 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b8229a934e2ad6b3666ff845bbe', 'height': 64, 'width': 64}, {'url': 'https://i.scdn.co/image/ab6775700000ee8529a934e2ad6b3666ff845bbe', 'height': 300, 'width': 300}], 'type': 'user', 'uri': 'spotify:user:grantdidway12345', 'followers': {'href': None, 'total': 40}}


In [12]:
stream = pd.read_csv('stream.csv')

## Part 1

Retrieving genre info for each artist in my history

In [18]:
artists = stream.groupby('artist name')['mins_played'].sum().reset_index()
artists.sort_values(by='mins_played', ascending=False,inplace=True)
artists

,artist name,mins_played
4492,"Slaughter Beach, Dog",13484.845167
1122,Cuco,6231.511933
1126,Current Joys,3011.730583
5294,Troye Sivan,2617.128367
2764,King Princess,2522.860717
...,...,...
2794,Kowloon,0.000000
4791,Teenage Fanclub,0.000000
3366,Midlake,0.000000
4949,The House of Love,0.000000


In [19]:
artists_with_genres = artists

# test
artist_name = "Adele"
genres = spotify.get_artist_genre(artist_name)
print("Genres for", artist_name, ":", genres)


# Handling for a list of artists
genres = []
artist_genres = {}
for index, artist in enumerate(artists_with_genres['artist name']):
    artist_genre = spotify.get_artist_genre(artist)
    genres.append(artist_genre)
    artist_genres[artist] = artist_genre
    if index % 100 == 0:
        print(index, artist)  # Show the progress

artists_with_genres['genres'] = genres
artists_with_genres


Genres for Adele : ['british soul', 'pop', 'pop soul', 'uk pop']
0 Slaughter Beach, Dog
100 Kevin Abstract
200 The Hails
300 WILLOW
400 ROLE MODEL
500 Natalie Cole
600 Little Jesus
700 Doc & Merle Watson
800 Mt. Joy
900 Junior Mance
1000 Mal Blum
1100 Odd Future
1200 mage tears
1300 Frederick Delius
1400 Brett Eldredge
1500 Sadurn
1600 The Red Pears
1700 Lo Mimieux
1800 Courtney Hartman
1900 Alonzo Gautier
2000 Matti Aalto
2100 Keyshia Cole
2200 Pitch Slapped
2300 Mirah
2400 Kristen Nichols
2500 Tunji Ige
2600 KAINA
2700 Tems
2800 FDVM
2900 Frances Quinlan
3000 DaniLeigh
3100 Un León Marinero
3200 Bill Withers
3300 Kay Adimon
3400 Maude Eldridge
3500 Ashnikko
3600 $NOT
3700 Vegyn
3800 I'm Glad It's You
3900 Pink Swan
4000 Taj Mahal
4100 Deorro
4200 Igor Stravinsky
4300 The Water Cycle
4400 Charley Pride
4500 CNCO
4600 M1RZA
4700 Def Leppard
4800 Badgirl$
4900 The Dubs
5000 Kainen Kellum
5100 Damon & Naomi
5200 Preoccupations
5300 John Splithoff
5400 Sayed Darwish
5500 Jacob Collier
560

,artist name,mins_played,genres
4492,"Slaughter Beach, Dog",13484.845167,"[alternative emo, bubblegrunge, emo, lo-fi emo..."
1122,Cuco,6231.511933,"[bedroom pop, dream pop]"
1126,Current Joys,3011.730583,"[lo-fi indie, pov: indie]"
5294,Troye Sivan,2617.128367,"[australian pop, pop, viral pop]"
2764,King Princess,2522.860717,"[alt z, indie pop, nyc pop]"
...,...,...,...
2794,Kowloon,0.000000,[]
4791,Teenage Fanclub,0.000000,"[alternative rock, c86, modern power pop, powe..."
3366,Midlake,0.000000,"[austindie, chamber pop, denton tx indie, mode..."
4949,The House of Love,0.000000,"[britpop, c86, dream pop, uk post-punk]"


In [21]:
artists_with_genres.to_csv('artists_with_genres.csv')

# Part 2
Retrieving audio features for each song in my history

In [21]:
songs = stream.groupby(['track name', 'artist name', 'album name', 'uri'])['mins_played'].sum().reset_index()
songs.rename(columns={'mins_played': 'total_mins'}, inplace=True)

songs.sort_values(by='total_mins', ascending=True, inplace=True)
songs.reset_index()
songs

,track name,artist name,album name,uri,total_mins
7227,Hands On,Kanye West,JESUS IS KING,spotify:track:7r6aR2f57resC5r93LwMfK,0.000000
15107,Sastanàqqàm,Tinariwen,Elwan,spotify:track:3qd6kWpXeX5SMvVuiS9qBY,0.000000
15097,Sappho,Frankie Cosmos,Next Thing,spotify:track:5DIGt883aHs3KqWSziy4gR,0.000000
15049,Sam,Meat Puppets,Forbidden Places,spotify:track:36UDI4VPxbHzTEuvrOTH7V,0.000000
6319,Friendship,Chris Stapleton,From A Room: Volume 2,spotify:track:2ZAqfeq7PMp7XA2IKhuJJ2,0.000000
...,...,...,...,...,...
1042,Amor de Siempre,Cuco,wannabewithu,spotify:track:6VZg3YsN5kKBwz39fPcIM1,404.622600
614,Acolyte,"Slaughter Beach, Dog",Birdie,spotify:track:2h0O3uXZW6ClktlgHWGpjy,463.891183
10505,Lo Que Siento,Cuco,Lo Que Siento,spotify:track:3oyf4dalm17kzTE8LJDbgR,481.645467
19042,Upper West Side,King Princess,Make My Bed,spotify:track:7vfaRUaFBDwENwa3tzJY64,485.726000


In [22]:
def process_audio_features(audio_features, songs):
    for track_features in audio_features:
        if track_features:
            track_uri = track_features['uri']
            # Find the corresponding row in the DataFrame
            index = songs.index[songs['uri'] == track_uri].tolist()[0]
            for feature, value in track_features.items():
                # Add each audio feature as a new column
                if feature not in songs.columns:
                    songs[feature] = np.nan
                songs.at[index, feature] = value


In [24]:
# Split URIs into smaller batches (20 URIs per batch)
uri_batches = [songs['uri'][i:i+20] for i in range(0, len(songs['uri']), 20)]
count = 0
# Process each batch with a delay between each batch
for batch in uri_batches:
    if count % 20 == 0:
        print(count)
    audio_features = spotify.get_audio_features_batch(batch)
    process_audio_features(audio_features, songs)
    time.sleep(2)  # Delay between batches to avoid hitting rate limits
    count+=1

# The 'songs' DataFrame will now have new columns for each audio feature


0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060


Max Retries reached


Rate limit exceeded, retrying in 5 seconds...


Max Retries reached


Rate limit exceeded, retrying in 10 seconds...


...

Max Retries reached


Rate limit exceeded, retrying in 40 seconds...


Max Retries reached


Rate limit exceeded, retrying in 80 seconds...
Failed to fetch audio features after maximum retries.


TypeError: 'NoneType' object is not iterable

In [33]:
songs.reset_index(inplace=True)
songs

,level_0,index,track name,artist name,album name,uri,total_mins,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,0,7227,Hands On,Kanye West,JESUS IS KING,spotify:track:7r6aR2f57resC5r93LwMfK,0.000000,0.514,0.393,1.0,...,0.000000,0.282,0.598,78.039,audio_features,7r6aR2f57resC5r93LwMfK,https://api.spotify.com/v1/tracks/7r6aR2f57res...,https://api.spotify.com/v1/audio-analysis/7r6a...,203174.0,4.0
1,1,15107,Sastanàqqàm,Tinariwen,Elwan,spotify:track:3qd6kWpXeX5SMvVuiS9qBY,0.000000,0.730,0.663,5.0,...,0.049700,0.105,0.428,98.519,audio_features,3qd6kWpXeX5SMvVuiS9qBY,https://api.spotify.com/v1/tracks/3qd6kWpXeX5S...,https://api.spotify.com/v1/audio-analysis/3qd6...,204600.0,4.0
2,2,15097,Sappho,Frankie Cosmos,Next Thing,spotify:track:5DIGt883aHs3KqWSziy4gR,0.000000,0.644,0.401,7.0,...,0.000000,0.132,0.459,95.354,audio_features,5DIGt883aHs3KqWSziy4gR,https://api.spotify.com/v1/tracks/5DIGt883aHs3...,https://api.spotify.com/v1/audio-analysis/5DIG...,112879.0,4.0
3,3,15049,Sam,Meat Puppets,Forbidden Places,spotify:track:36UDI4VPxbHzTEuvrOTH7V,0.000000,0.396,0.889,6.0,...,0.103000,0.320,0.609,193.066,audio_features,36UDI4VPxbHzTEuvrOTH7V,https://api.spotify.com/v1/tracks/36UDI4VPxbHz...,https://api.spotify.com/v1/audio-analysis/36UD...,192307.0,4.0
4,4,6319,Friendship,Chris Stapleton,From A Room: Volume 2,spotify:track:2ZAqfeq7PMp7XA2IKhuJJ2,0.000000,0.680,0.334,0.0,...,0.000491,0.112,0.483,129.526,audio_features,2ZAqfeq7PMp7XA2IKhuJJ2,https://api.spotify.com/v1/tracks/2ZAqfeq7PMp7...,https://api.spotify.com/v1/audio-analysis/2ZAq...,265173.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21503,21503,1042,Amor de Siempre,Cuco,wannabewithu,spotify:track:6VZg3YsN5kKBwz39fPcIM1,404.622600,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21504,21504,614,Acolyte,"Slaughter Beach, Dog",Birdie,spotify:track:2h0O3uXZW6ClktlgHWGpjy,463.891183,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21505,21505,10505,Lo Que Siento,Cuco,Lo Que Siento,spotify:track:3oyf4dalm17kzTE8LJDbgR,481.645467,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21506,21506,19042,Upper West Side,King Princess,Make My Bed,spotify:track:7vfaRUaFBDwENwa3tzJY64,485.726000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#Check for na values

for i in songs.columns:
    print(i, songs[i].isna().sum())

track name 0
artist name 0
album name 0
uri 0
total_mins 0
danceability 155
energy 155
key 155
loudness 155
mode 155
speechiness 155
acousticness 155
instrumentalness 155
liveness 155
valence 155
tempo 155
type 155
id 155
track_href 155
analysis_url 155
duration_ms 155
time_signature 155


The last calls reached the API rate limit so I saved the one's that were missed and retrieved the audio features using a different developer account

In [45]:
leftovers = songs.iloc[-148:]
leftovers.to_csv('leftovers.csv')
leftovers

,level_0,index,track name,artist name,album name,uri,total_mins,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
21360,21360,16328,Soon or Never,Punch Brothers,Who's Feeling Young Now?,spotify:track:04gmJvY5T9duH3eFXwPxV7,140.015800,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21361,21361,3797,Corduroy Dreams,Rex Orange County,Bcos U Will Never B Free,spotify:track:02tvc9CFnTyHuSRlGeNv9w,140.275483,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21362,21362,9971,Last Snowstorm of the Year,Hippo Campus,Last Snowstorm of the Year,spotify:track:4p32IK311NkLVtCwTTBG26,140.616283,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21363,21363,19339,Waiting for You,Verzache,D97,spotify:track:1olz3HctPa2yXmSniaCNLN,140.914733,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21364,21364,748,Alabina - Original 1996,Alabina,The Album of Alabina & Los Niños de Sara,spotify:track:4c9PMOg5YtNzU7o8CYTOmb,140.953633,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21503,21503,1042,Amor de Siempre,Cuco,wannabewithu,spotify:track:6VZg3YsN5kKBwz39fPcIM1,404.622600,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21504,21504,614,Acolyte,"Slaughter Beach, Dog",Birdie,spotify:track:2h0O3uXZW6ClktlgHWGpjy,463.891183,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21505,21505,10505,Lo Que Siento,Cuco,Lo Que Siento,spotify:track:3oyf4dalm17kzTE8LJDbgR,481.645467,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21506,21506,19042,Upper West Side,King Princess,Make My Bed,spotify:track:7vfaRUaFBDwENwa3tzJY64,485.726000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Below I concatenate the dataframes and save the final as songs_with_audio_features.csv

In [50]:
leftovers = pd.read_csv('leftovers.csv')
songs1 = songs[~songs['uri'].isin(leftovers['uri'])]
songs1


,level_0,index,track name,artist name,album name,uri,total_mins,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,0,7227,Hands On,Kanye West,JESUS IS KING,spotify:track:7r6aR2f57resC5r93LwMfK,0.000000,0.514,0.393,1.0,...,0.000000,0.2820,0.5980,78.039,audio_features,7r6aR2f57resC5r93LwMfK,https://api.spotify.com/v1/tracks/7r6aR2f57res...,https://api.spotify.com/v1/audio-analysis/7r6a...,203174.0,4.0
1,1,15107,Sastanàqqàm,Tinariwen,Elwan,spotify:track:3qd6kWpXeX5SMvVuiS9qBY,0.000000,0.730,0.663,5.0,...,0.049700,0.1050,0.4280,98.519,audio_features,3qd6kWpXeX5SMvVuiS9qBY,https://api.spotify.com/v1/tracks/3qd6kWpXeX5S...,https://api.spotify.com/v1/audio-analysis/3qd6...,204600.0,4.0
2,2,15097,Sappho,Frankie Cosmos,Next Thing,spotify:track:5DIGt883aHs3KqWSziy4gR,0.000000,0.644,0.401,7.0,...,0.000000,0.1320,0.4590,95.354,audio_features,5DIGt883aHs3KqWSziy4gR,https://api.spotify.com/v1/tracks/5DIGt883aHs3...,https://api.spotify.com/v1/audio-analysis/5DIG...,112879.0,4.0
3,3,15049,Sam,Meat Puppets,Forbidden Places,spotify:track:36UDI4VPxbHzTEuvrOTH7V,0.000000,0.396,0.889,6.0,...,0.103000,0.3200,0.6090,193.066,audio_features,36UDI4VPxbHzTEuvrOTH7V,https://api.spotify.com/v1/tracks/36UDI4VPxbHz...,https://api.spotify.com/v1/audio-analysis/36UD...,192307.0,4.0
4,4,6319,Friendship,Chris Stapleton,From A Room: Volume 2,spotify:track:2ZAqfeq7PMp7XA2IKhuJJ2,0.000000,0.680,0.334,0.0,...,0.000491,0.1120,0.4830,129.526,audio_features,2ZAqfeq7PMp7XA2IKhuJJ2,https://api.spotify.com/v1/tracks/2ZAqfeq7PMp7...,https://api.spotify.com/v1/audio-analysis/2ZAq...,265173.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21355,21355,10519,Locket,Crumb,Locket,spotify:track:0bxcUgWlOURkU6lZt4zog0,138.933450,0.393,0.481,1.0,...,0.730000,0.0869,0.0763,98.462,audio_features,0bxcUgWlOURkU6lZt4zog0,https://api.spotify.com/v1/tracks/0bxcUgWlOURk...,https://api.spotify.com/v1/audio-analysis/0bxc...,318379.0,4.0
21356,21356,19919,Where'd All the Time Go?,Dr. Dog,"Shame, Shame",spotify:track:0UV5zxRMz6AO4ZwUOZNIKI,139.222567,0.454,0.820,2.0,...,0.000969,0.1150,0.5750,166.303,audio_features,0UV5zxRMz6AO4ZwUOZNIKI,https://api.spotify.com/v1/tracks/0UV5zxRMz6AO...,https://api.spotify.com/v1/audio-analysis/0UV5...,234800.0,4.0
21357,21357,7374,Hazy Miss Daisy,Kid Bloom,A Different State of Mind,spotify:track:6b8nz7zFlAeNACOA1lVYG7,139.592317,0.747,0.368,8.0,...,0.008700,0.1260,0.7730,76.008,audio_features,6b8nz7zFlAeNACOA1lVYG7,https://api.spotify.com/v1/tracks/6b8nz7zFlAeN...,https://api.spotify.com/v1/audio-analysis/6b8n...,217105.0,4.0
21358,21358,4613,Do You Understand (What Has Happened to You),"Slaughter Beach, Dog",At the Moonbase,spotify:track:16bB8g67GwAhSBewSTP00a,139.856567,0.662,0.679,9.0,...,0.000550,0.0626,0.5180,97.014,audio_features,16bB8g67GwAhSBewSTP00a,https://api.spotify.com/v1/tracks/16bB8g67GwAh...,https://api.spotify.com/v1/audio-analysis/16bB...,283536.0,4.0


In [51]:
songs_with_audio_features = pd.concat([songs1, leftovers], ignore_index=True)

In [52]:
songs_with_audio_features

,level_0,index,track name,artist name,album name,uri,total_mins,danceability,energy,key,...,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,Unnamed: 0.1,Unnamed: 0
0,0,7227,Hands On,Kanye West,JESUS IS KING,spotify:track:7r6aR2f57resC5r93LwMfK,0.000000,0.514,0.393,1.0,...,0.598,78.039,audio_features,7r6aR2f57resC5r93LwMfK,https://api.spotify.com/v1/tracks/7r6aR2f57res...,https://api.spotify.com/v1/audio-analysis/7r6a...,203174.0,4.0,NaN,NaN
1,1,15107,Sastanàqqàm,Tinariwen,Elwan,spotify:track:3qd6kWpXeX5SMvVuiS9qBY,0.000000,0.730,0.663,5.0,...,0.428,98.519,audio_features,3qd6kWpXeX5SMvVuiS9qBY,https://api.spotify.com/v1/tracks/3qd6kWpXeX5S...,https://api.spotify.com/v1/audio-analysis/3qd6...,204600.0,4.0,NaN,NaN
2,2,15097,Sappho,Frankie Cosmos,Next Thing,spotify:track:5DIGt883aHs3KqWSziy4gR,0.000000,0.644,0.401,7.0,...,0.459,95.354,audio_features,5DIGt883aHs3KqWSziy4gR,https://api.spotify.com/v1/tracks/5DIGt883aHs3...,https://api.spotify.com/v1/audio-analysis/5DIG...,112879.0,4.0,NaN,NaN
3,3,15049,Sam,Meat Puppets,Forbidden Places,spotify:track:36UDI4VPxbHzTEuvrOTH7V,0.000000,0.396,0.889,6.0,...,0.609,193.066,audio_features,36UDI4VPxbHzTEuvrOTH7V,https://api.spotify.com/v1/tracks/36UDI4VPxbHz...,https://api.spotify.com/v1/audio-analysis/36UD...,192307.0,4.0,NaN,NaN
4,4,6319,Friendship,Chris Stapleton,From A Room: Volume 2,spotify:track:2ZAqfeq7PMp7XA2IKhuJJ2,0.000000,0.680,0.334,0.0,...,0.483,129.526,audio_features,2ZAqfeq7PMp7XA2IKhuJJ2,https://api.spotify.com/v1/tracks/2ZAqfeq7PMp7...,https://api.spotify.com/v1/audio-analysis/2ZAq...,265173.0,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21503,21503,1042,Amor de Siempre,Cuco,wannabewithu,spotify:track:6VZg3YsN5kKBwz39fPcIM1,404.622600,0.565,0.642,0.0,...,0.539,122.989,audio_features,6VZg3YsN5kKBwz39fPcIM1,https://api.spotify.com/v1/tracks/6VZg3YsN5kKB...,https://api.spotify.com/v1/audio-analysis/6VZg...,321951.0,4.0,143.0,21503.0
21504,21504,614,Acolyte,"Slaughter Beach, Dog",Birdie,spotify:track:2h0O3uXZW6ClktlgHWGpjy,463.891183,0.790,0.476,9.0,...,0.774,123.977,audio_features,2h0O3uXZW6ClktlgHWGpjy,https://api.spotify.com/v1/tracks/2h0O3uXZW6Cl...,https://api.spotify.com/v1/audio-analysis/2h0O...,316468.0,4.0,144.0,21504.0
21505,21505,10505,Lo Que Siento,Cuco,Lo Que Siento,spotify:track:3oyf4dalm17kzTE8LJDbgR,481.645467,0.326,0.685,10.0,...,0.409,86.882,audio_features,3oyf4dalm17kzTE8LJDbgR,https://api.spotify.com/v1/tracks/3oyf4dalm17k...,https://api.spotify.com/v1/audio-analysis/3oyf...,312000.0,5.0,145.0,21505.0
21506,21506,19042,Upper West Side,King Princess,Make My Bed,spotify:track:7vfaRUaFBDwENwa3tzJY64,485.726000,0.635,0.496,9.0,...,0.733,141.863,audio_features,7vfaRUaFBDwENwa3tzJY64,https://api.spotify.com/v1/tracks/7vfaRUaFBDwE...,https://api.spotify.com/v1/audio-analysis/7vfa...,211413.0,4.0,146.0,21506.0


In [54]:
songs_with_audio_features.drop(['level_0', 'index', 'Unnamed: 0.1', 'Unnamed: 0'], axis=1)


,track name,artist name,album name,uri,total_mins,danceability,energy,key,loudness,mode,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Hands On,Kanye West,JESUS IS KING,spotify:track:7r6aR2f57resC5r93LwMfK,0.000000,0.514,0.393,1.0,-8.720,0.0,...,0.000000,0.282,0.598,78.039,audio_features,7r6aR2f57resC5r93LwMfK,https://api.spotify.com/v1/tracks/7r6aR2f57res...,https://api.spotify.com/v1/audio-analysis/7r6a...,203174.0,4.0
1,Sastanàqqàm,Tinariwen,Elwan,spotify:track:3qd6kWpXeX5SMvVuiS9qBY,0.000000,0.730,0.663,5.0,-7.778,0.0,...,0.049700,0.105,0.428,98.519,audio_features,3qd6kWpXeX5SMvVuiS9qBY,https://api.spotify.com/v1/tracks/3qd6kWpXeX5S...,https://api.spotify.com/v1/audio-analysis/3qd6...,204600.0,4.0
2,Sappho,Frankie Cosmos,Next Thing,spotify:track:5DIGt883aHs3KqWSziy4gR,0.000000,0.644,0.401,7.0,-6.782,1.0,...,0.000000,0.132,0.459,95.354,audio_features,5DIGt883aHs3KqWSziy4gR,https://api.spotify.com/v1/tracks/5DIGt883aHs3...,https://api.spotify.com/v1/audio-analysis/5DIG...,112879.0,4.0
3,Sam,Meat Puppets,Forbidden Places,spotify:track:36UDI4VPxbHzTEuvrOTH7V,0.000000,0.396,0.889,6.0,-10.994,0.0,...,0.103000,0.320,0.609,193.066,audio_features,36UDI4VPxbHzTEuvrOTH7V,https://api.spotify.com/v1/tracks/36UDI4VPxbHz...,https://api.spotify.com/v1/audio-analysis/36UD...,192307.0,4.0
4,Friendship,Chris Stapleton,From A Room: Volume 2,spotify:track:2ZAqfeq7PMp7XA2IKhuJJ2,0.000000,0.680,0.334,0.0,-11.375,1.0,...,0.000491,0.112,0.483,129.526,audio_features,2ZAqfeq7PMp7XA2IKhuJJ2,https://api.spotify.com/v1/tracks/2ZAqfeq7PMp7...,https://api.spotify.com/v1/audio-analysis/2ZAq...,265173.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21503,Amor de Siempre,Cuco,wannabewithu,spotify:track:6VZg3YsN5kKBwz39fPcIM1,404.622600,0.565,0.642,0.0,-7.210,1.0,...,0.358000,0.138,0.539,122.989,audio_features,6VZg3YsN5kKBwz39fPcIM1,https://api.spotify.com/v1/tracks/6VZg3YsN5kKB...,https://api.spotify.com/v1/audio-analysis/6VZg...,321951.0,4.0
21504,Acolyte,"Slaughter Beach, Dog",Birdie,spotify:track:2h0O3uXZW6ClktlgHWGpjy,463.891183,0.790,0.476,9.0,-10.674,1.0,...,0.094200,0.113,0.774,123.977,audio_features,2h0O3uXZW6ClktlgHWGpjy,https://api.spotify.com/v1/tracks/2h0O3uXZW6Cl...,https://api.spotify.com/v1/audio-analysis/2h0O...,316468.0,4.0
21505,Lo Que Siento,Cuco,Lo Que Siento,spotify:track:3oyf4dalm17kzTE8LJDbgR,481.645467,0.326,0.685,10.0,-5.576,1.0,...,0.000000,0.427,0.409,86.882,audio_features,3oyf4dalm17kzTE8LJDbgR,https://api.spotify.com/v1/tracks/3oyf4dalm17k...,https://api.spotify.com/v1/audio-analysis/3oyf...,312000.0,5.0
21506,Upper West Side,King Princess,Make My Bed,spotify:track:7vfaRUaFBDwENwa3tzJY64,485.726000,0.635,0.496,9.0,-8.858,1.0,...,0.006340,0.081,0.733,141.863,audio_features,7vfaRUaFBDwENwa3tzJY64,https://api.spotify.com/v1/tracks/7vfaRUaFBDwE...,https://api.spotify.com/v1/audio-analysis/7vfa...,211413.0,4.0


In [55]:
#Save progress
songs_with_audio_features.to_csv('songs_with_audio_features.csv')